# For Getting channels log

## Import modules

In [ ]:
import os
from slack_sdk import WebClient
import pandas as pd
import numpy as np
import datetime
from collections import OrderedDict
import openpyxl
import urllib

## WebApp settings

In [ ]:
proxy = urllib.request.getproxies().get('http')
print(proxy)
client = WebClient(token=os.environ['TEST_CH_BOT_TOKEN'], proxy=proxy)
uclient = WebClient(token=os.environ['TEST_CH_USER_TOKEN'], proxy=proxy)

## get messages (without replies) in specified channel 

### get channels (chs) ids and names in your workspace
#### retrive wholl info of chs (for backup raw data)

In [ ]:
ws_info = uclient.conversations_list()
ws_info_ch = ws_info.get('channels')
df_ch_info = pd.json_normalize(ws_info_ch)
print(df_ch_info.columns)

#### extract chs ids and names

In [ ]:
ch_id_name = []
for i in zip(df_ch_info['id'], df_ch_info['name']):
    ch_id_name.append(i)

df_ch_id_name = df_ch_info[['id', 'name']]
df_ch_id_name

---
### specify channel for retriving log

In [ ]:
# example: specify a channel named 'devslacksdk'
ch_idx = 1 # select number on the left of 'id' column above table

ch_id = df_ch_id_name['id'][ch_idx]
ch_name = df_ch_id_name['name'][ch_idx]
print('Specified channel: '+ch_name+', ch_id: '+ch_id)

### get messages log for specified ch

#### retrive raw data and create table of messages log

In [ ]:
ch_log = uclient.conversations_history(channel=ch_id) # it is raw data without replies
ch_ms_log = ch_log['messages']

df_ch_ms_log = pd.json_normalize(ch_ms_log)
list_log = []

# slack API can retrive only 100 messages at once, so following steps are for retriving over 100.
i = 0
if ch_log['has_more'] == True:
    while ch_log['has_more'] == True:
        ch_log = uclient.conversations_history(channel=ch_id, cursor=ch_log['response_metadata']['next_cursor'])
        ch_ms_log_n = ch_log['messages']
        list_log.append(ch_log['messages'])
        
        df_ch_ms_log_n = pd.json_normalize(ch_ms_log_n)
        df_ch_ms_log = df_ch_ms_log.append(df_ch_ms_log_n)
        i += 1
print(i)
flat = [x for row in list_log for x in row]
ch_ms_log.extend(flat) # ch_ms_log can be stored over 100 logs
print(len(ch_ms_log)) # num of messages
df_ch_ms_log.columns

#### select info what you want to retrive and format table

In [ ]:
reindex_col = ['type', 'thread_ts', 'ts', 'user', 'text', 'reply_count', 'reply_users_count', 'topic']
df_ch_ms_log = df_ch_ms_log.reindex(columns=reindex_col)
df_ch_ms_log

### (in MESSAGES) retrive attached files and create table contains file name and download url

In [ ]:
files = []

for i in df_ch_ms_log.index:
    if 'files' in ch_ms_log[i].keys():
        files.append(ch_ms_log[i]['files'][0])
    else:
        files.append({'name': 'None', 'url_private_download': 'None'})  

df_ch_files_ms = pd.json_normalize(files)
df_ch_files_ms = df_ch_files_ms[['name', 'url_private_download']]
df_ch_files_ms = df_ch_files_ms.rename(columns={'name': 'FileName', 'url_private_download': 'FileURL'})
df_ch_files_ms

### combine MESSAGE table with file rep table and sort by ts

In [ ]:
df_ch_ms_log = pd.concat([df_ch_ms_log, df_ch_files_ms], axis=1).sort_values('ts').reset_index().drop(columns='index')
df_ch_ms_log

---
## get replies to associate with each replies by parent messages

### search messages including replies and retrive reps with parent messages

In [ ]:
threads = []
for i in df_ch_ms_log.index:
    if np.isnan(df_ch_ms_log['reply_count'][i]) == False:
        thr = uclient.conversations_replies(channel=ch_id, ts=df_ch_ms_log['thread_ts'][i])
        thr_ms = thr.get('messages')
        threads.append(thr_ms)

In [ ]:
reps = []
for i in range(len(threads)):
    rep = threads[i]
    for j in range(len(rep)):
        reps.append(rep[j])
print(len(reps))

### following steps for create table contains messages and reps
#### create reply table
#### (in REPLIES) retrive attached files and create table contains file name and download url
#### combine REPLY table with FILE table and sort by timestamp

In [ ]:
if len(reps) != 0:
    # create reply table
    reindex_col = ['type', 'thread_ts', 'ts', 'user', 'text', 'reply_count', 'reply_users_count', 'topic']
    df_ch_rep = pd.json_normalize(reps).reindex(columns=reindex_col)

    
    # (in REPLIES) retrive attached files and create table contains file name and download url
    files_rep = []
    for i in range(len(reps)):
        if 'files' in reps[i].keys():
            files_rep.append(reps[i]['files'][0])
        else:
            files_rep.append({'name': 'None', 'url_private_download': 'None'})

    df_files_rep = pd.json_normalize(files_rep)
    df_files_rep = df_files_rep[['name', 'url_private_download']]
    df_files_rep = df_files_rep.rename(columns={'name': 'FileName', 'url_private_download': 'FileURL'})
    print(df_files_rep)

    # combine REPLY table with FILE table and sort by thread_ts
    df_ch_rep_log = pd.concat([df_ch_rep, df_files_rep], axis=1).sort_values('thread_ts').reset_index().drop(columns='index')
    df_ch_rep_log['type']='thread'
    print(df_ch_rep_log)

    
    # combine MESSAGE with REPLY 
    df_ch_log = pd.concat([df_ch_ms_log, df_ch_rep_log])
    # remove duplicated parent messages by drop_duplicates
    df_ch_log = df_ch_log.drop_duplicates(subset = ['text', 'ts'], keep='last').reset_index().drop(columns='index')


else:
    df_ch_log = df_ch_ms_log
df_ch_log

## shape table

### convert dtypes of ts and thread_ts, from str to float, and replace NaN to 'None'

In [ ]:
df_ch_log_astype = df_ch_log.astype({'ts': float, 'thread_ts': float}).fillna('None')
df_ch_log_astype

### replace UNIX DATE to formatted one and separate ts to date and time (for multiindex)

In [ ]:
date = []
time = []
for i in df_ch_log_astype.index:
    if (type(df_ch_log_astype['ts'][i]) == np.float64 or 
        type(df_ch_log_astype['ts'][i]) == float):
        # create list of date
        dt_raw = datetime.date.fromtimestamp(df_ch_log_astype['ts'][i])
        dt = dt_raw.strftime('%a, %b %d, %Y')
        date.append(dt)
        print('finished formatting ts to date')        
        
        # create list of time
        ti_raw = datetime.datetime.fromtimestamp(df_ch_log_astype['ts'][i])
        ti = ti_raw.strftime('%H:%M')
        time.append(ti)
        print('finished formatting ts to time') 
    
    # format thread_ts to datetime
    if (type(df_ch_log_astype['thread_ts'][i]) == np.float64 or 
        type(df_ch_log_astype['thread_ts'][i]) == float):
        dtime = datetime.datetime.fromtimestamp(df_ch_log_astype['thread_ts'][i])
        df_ch_log_astype.iloc[i, 1] = dtime.strftime('%H:%M, %a, %b %d, %Y')
        print ('finished formatting thread_ts')
    
    else:
        print('thread_ts is not found')

df_datetime = pd.DataFrame({'date': date, 
                            'time': time})
df_ch_log_frt = pd.concat([df_datetime, df_ch_log_astype],axis=1).drop(columns='ts')
df_ch_log_frt

### get users info

In [ ]:
users_info = uclient.users_list().get('members')
df_users_info = pd.json_normalize(users_info)

# create list of user_id and names
user_id_names = []
for i in zip(df_users_info['id'], df_users_info['real_name']):
    user_id_names.append(i)
df_user_id_names = pd.DataFrame(user_id_names,columns=['id', 'real_name']).fillna('deleted user').astype({'real_name': str})
df_user_id_names

### replace user id in 'names' column to 'real_name'

In [ ]:
for i in df_ch_log_frt.index:
    for j in range(len(user_id_names)):
        if df_ch_log_frt.user[i] == df_user_id_names['id'][j]:
            df_ch_log_frt.user[i] = df_user_id_names['real_name'][j]

### replace user id in 'text' column to 'real_name' (maybe able to merge above cell?)

In [ ]:
for i in df_ch_log_frt.index:
    for j in df_user_id_names.index:
        if df_user_id_names['id'][j] in df_ch_log_frt['text'][i]:
            df_ch_log_frt['text'][i] = df_ch_log_frt['text'][i].replace(df_user_id_names['id'][j], df_user_id_names['real_name'][j])
        else:
            continue

### for only uploading file (None of text fields)

In [ ]:
for i in df_ch_log_frt.index:
    if df_ch_log_frt.text[i] == '':
        df_ch_log_frt.text[i] = 'attached file(s) only'
    else:
        continue

### check result of formatting and replacing

In [ ]:
df_ch_log_frt

### multiIndex

In [ ]:
df_ch_log_frt_mi = df_ch_log_frt.set_index(['type', 'thread_ts', 'date', 'time'])
df_ch_log_frt_mi

## export files

### setting *you can custom something (year, path, etc.)

In [ ]:
backup_date = datetime.date.today()
backup_date = backup_date.strftime('%y%m%d')

# DO NOT FORGET MODIFY YEAR (year of current backup workspace) 
tgt_ws_year = 2022
print(backup_date)
print(ch_name)

In [ ]:
path_ch = '../channel/'
if os.path.exists(path_ch) == False:
    %mkdir path_ch
else:
    print('directory already exists')
    
path_ch_full = '../ch_full_log/'
if os.path.exists(path_ch_full) == False:
    %mkdir path_ch_full
else:
    print('directory already exists')
    
path_YEAR = str(tgt_ws_year)
if os.path.exists(path_ch+path_YEAR) == False:
    %mkdir path_ch+path_YEAR
else:
    print('directory already exists')

if os.path.exists(path_ch_full+path_YEAR) == False:
    %mkdir path_ch_full+path_YEAR
else:
    print('directory already exists')

### export excel file (for remaining multiIndex and encoding automatically)

In [ ]:
df_ch_log_frt_mi.to_excel(path_ch+path_YEAR+'/log_ch_'+path_YEAR+'_'+ch_name+'_'+backup_date+'.xlsx')

### export raw file (json format)

#### not included replies

In [ ]:
with open(path_ch_full+path_YEAR+'/Raw_ch_log_NOrep_'+path_YEAR+'_'+ch_name+'_'+backup_date+'.dat', 'w') as f:
    print(ch_log, file=f) 

#### replies data

In [ ]:
with open(path_ch_full+path_YEAR+'/Raw_ch_rep_log_'+path_YEAR+'_'+ch_name+'_'+backup_date+'.dat', 'w') as f:
    print(thr, file=f) 